# Optimizing Model Hosting and Inference Costs

## Real-time inference versus batch inference
#### SageMaker provides two ways to obtain inferences:

- Real-time inference lets you get a single inference per request, or a small number of inferences, with very low latency from a live inference endpoint.
- Batch inference lets you get a large number of inferences from a batch processing job.

#### Batch inference is more efficient and more cost-effective. Use it whenever your inference requirements allow. We'll explore batch inference first, and then pivot to real-time inference.